In [81]:
import numpy as np
import pandas as pd
import os
import requests
from datetime import datetime
from bs4 import BeautifulSoup, SoupStrainer
import requests
from io import StringIO
import re
import calendar as cl
import locale
import json

In [155]:
blob_container_name = 'general'                                             # container name

blob_relative_path_raw = 'raw/mercado_potencial/anp/tancagem_terminais/'    # relative folder path
blob_relative_path_enriched = 'enriched/mercado_potencial/'                 # relative folder path

linked_service_raw = 'LS_ADLS_RAW_01'                                       # linked service name
linked_service_enriched = 'LS_ADLS_ENRICHED_01'                             # linked service name


ls_raw = mssparkutils.credentials.getPropertiesAll(linked_service_raw)
ls_enriched = mssparkutils.credentials.getPropertiesAll(linked_service_enriched)

converter_dic_raw = json.loads(ls_raw)
converter_dic_enriched = json.loads(ls_enriched)

#coletando o endpoint
end_point_raw = (converter_dic_raw['Endpoint'].split("/"))[2]
end_point_enriched = (converter_dic_enriched['Endpoint'].split("/"))[2] 
#Utilizado na leitura via metodo mssparkutils
abfss_path_raw = 'abfss://%s@%s/%s' % (blob_container_name, end_point_raw, blob_relative_path_raw)
abfss_path_enriched = 'abfss://%s@%s/%s' % (blob_container_name, end_point_enriched, blob_relative_path_enriched)

In [4]:
url = 'https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/tancagem-do-abastecimento-nacional-de-combustiveis'
dir_destino = 'dados/anp/'
nm_dado_sv= 'tancagem_terminais'
tipo = '.csv'
meses = {'janeiro': '01', 'fevereiro': '02', 'março'   : '03', 'marco'   : '03', 'mar'   : '03',
         'abril'  : '04', 'maio'     : '05', 'junho'   : '06',
         'julho'  : '07', 'agosto'   : '08', 'setembro': '09',
         'outubro': '10', 'novembro' : '11', 'dezembro': '12'}

In [5]:
soup = BeautifulSoup(requests.get(url).text,'html.parser',
                     parse_only=SoupStrainer('a'))

# dento da url pegar todos os elementos que tenham a tag html 'href' (links)
links = []
for link in soup:
    if link.has_attr('href'):
        for file_type in tipo:        
            if link['href'].endswith(tipo): # filtro end tipo
                links.append(link['href'])

In [7]:
# In[4]: Unificando links

links = np.unique(links)
arquivo = ['/'.join(y) for y in [x.split('/')[-2:] for x in links]]


In [42]:
def name_to_save(ls_links = links , nm_suf_sv = nm_dado_sv , tipo_sv = tipo):
    try:
        locale.setlocale(locale.LC_ALL, "pt_br")                                        # Auxilia nos nomes dos meses em [PT-BR]
    except:
        locale.setlocale(locale.LC_ALL, "pt_BR")

    # PADRÕES DO regex:
    date_pattern = '(\d{2,4} \d{1,2} \d{1,2})'                                          # data tipo %Y %m %d

    month_pattern = ['[{}{}]{}(?:{})?'.format(cl.month_name[i][0].upper(),              # nome mês em portugues
                                              cl.month_name[i][0].lower(),
                                              cl.month_name[i][1:3],
                                              cl.month_name[i][3:] ) for i in range(1,13)]
    month_pattern = '(?:'+'|'.join(month_pattern)+')'

    year_pattern = ''.join('\d{4}(?!\d)') # ou \d{2,4}(?!\d)\s*                         # padrão para ano

    nomes = [z.replace('_',' ') for \
                z in ['_'.join(y) for \
                    y in [x.split('/')[-2:] for \
                        x in ls_links]]]                                                # coleta os nomes a serem trabalhados

    n_data = []
    for x in nomes:
        if re.findall(date_pattern, x):                                                   # teste na data completa
            aux = ''.join(re.findall(date_pattern, x)).replace(' ','')
            n_data.append(aux)
        else:
            ano = re.findall(year_pattern, x)[0] \
                if re.findall(year_pattern, x) \
                    else '0000'     # sem ano                                           # operador ternário para o ano
            mes = meses[re.findall(month_pattern, x)[0]] \
                if re.findall(month_pattern, x) \
                    else '00'       # sem mes                                           # operador ternário para o mes
            n_data.append(ano+mes+'01')
    n_data = [(x +'-'+nm_suf_sv + tipo_sv) for x in n_data]
    return(n_data)


In [156]:
for i in range(len(arquivo)):
    file_name = abfss_path_raw + name_to_save(links)[i]
    url_ = [y for y in links if y.__contains__(arquivo[i])]
    df = requests.get(url=url_[0])
    try:
        df = df.content.decode('utf-8')
    except:
        df = df.content.decode('ISO-8859-1') # Arquivos podendo ter falhas no utf-8
    
    df = pd.read_csv(StringIO(df))
    df.to_csv(file_name, index = False, encoding='utf-8',
                storage_options = {'linked_service':linked_service_raw})
    print(name_to_save(links)[i], ' Salvo com sucesso!  ✔')

In [ ]:
# arquivos = [i.path for i in mssparkutils.fs.ls(abfss_path_raw) if i.path.endswith('tancagem_terminais.csv')]
# arquivos

In [ ]:
# pd.read_csv(arquivos[-1],encoding='utf-8')

In [ ]:
locale.setlocale(locale.LC_ALL, '')